In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
data = pd.read_excel("final_data_aqi.xlsx")


In [ ]:
data.head()

In [ ]:
data.drop("Unnamed: 0", axis = 1, inplace= True)
data.head()

In [ ]:
#Independent Variable
X = data.iloc[:, :-1]
Y= data.iloc[:, -1]

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
#Train Test Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)

In [ ]:
#Normalizing
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_xtrain = sc.fit_transform(x_train)
sc_xtest = sc.fit_transform(x_test)

In [ ]:
#Building ANN
from keras.models import Sequential
from keras.layers import Dense, Activation, ReLU, Dropout, LeakyReLU

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#Initializing the ANN
regressor = Sequential()
#Adding the input Layer 
regressor.add(Dense(units =128, kernel_initializer = "normal", input_dim = sc_xtrain.shape[1]))

In [ ]:
#Hidden Layer
regressor.add(Dense(units =256, kernel_initializer = "normal",activation = "relu"))
regressor.add(Dense(units =256, kernel_initializer = "normal",activation = "relu"))
regressor.add(Dense(units =256, kernel_initializer = "normal",activation = "relu"))

In [ ]:
#Adding Output Layer
regressor.add(Dense(units = 1, kernel_initializer = "normal", activation = "linear"))

In [ ]:
regressor.summary()

In [ ]:
#Compile ANN
regressor.compile(optimizer="adam", loss = "mean_squared_error", metrics = ["mean_squared_error"])

In [ ]:
model_history = regressor.fit(sc_xtrain, y_train,validation_split = 0.33,batch_size=15, epochs=100)

In [ ]:
ann_pred = regressor.predict(x_test)
sns.distplot(y_test.values.reshape(-1,1)- ann_pred)

In [ ]:
plt.scatter(y_test, ann_pred)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print("ACCURACY:", r2_score(y_test, ann_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, ann_pred)))
print("MSE:", mean_squared_error(y_test, ann_pred))

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.activations import relu, sigmoid
from keras.layers import Activation

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(nodes, input_dim = x_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units = 1, kernel_initializer="glorot_uniform", activation = "linear"))
    model.compile(optimizer = "Adam", loss = "mean_squared_error", metrics = ["accuracy"])
    return model  

In [ ]:
model = KerasRegressor(build_fn=create_model, verbose = 0)

In [ ]:
layers = [[20],[40,20], [45,30,15]]
activations = ["LeakyRelu"]
param_grid = dict(layers = layers, activation = activations, batch_size = [128, 256], epochs = [30,35,40])
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv =5)
grid_result = grid.fit(sc_xtrain, y_train)